# Unit 5 - Financial Planning

In [6]:
#Mick Charbonneau
#9/25/23
#Inspiration from UTOR Bootcamp Ressources & Activity + ChatGPT
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import yfinance as yf
import MCSimulation
import json
from PyForecastTools impo
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'MCSimulation'

In [ ]:
# Load .env enviroment variables
from dotenv import load_dotenv,find_dotenv # a more acurate line of code to locate .env file
load_dotenv(find_dotenv(r"C:\Users\MK\Desktop\python-homework\python notes\example.env")) #Location of my .env file

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

#Creating Alpaca API Object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

## Part 1 - Personal Finance Planner

### Collect Crypto Prices Using the `requests` Library

In [ ]:
# Set current amount of crypto assets
btc_holding = 1.2  
eth_holding = 5.3  

In [ ]:
# Crypto API URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=CAD" # Link to BTC API
btc_url= btc_url+"?format=json" # Format link to JSON format
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=CAD" # Link to ETH API
eth_url= eth_url+"?format=json" # Format link to JSON format

In [ ]:
# current BTC price
btc_data=requests.get("https://api.alternative.me/v2/ticker/Bitcoin/?convert=CAD") # New variable to contain BTC link and using the request command to retrive the link
data1=btc_data.json() # Reads the data in a JSON format
btc_price=data1["data"]["1"]["quotes"]["CAD"]["price"] # Specifying where im grabbing BTC price

#current ETH price
eth_data=requests.get("https://api.alternative.me/v2/ticker/Ethereum/?convert=CAD") # New variable to contain ETH link and using the request command to retrive the link
data2=eth_data.json() # Reads the data in a JSON format
eth_price=data2["data"]["1027"]["quotes"]["CAD"]["price"] # Specifying where im grabbing ETH price


# current value of my crpto
total_btc_price= btc_holding*btc_price # Current Price of BTC * Number of BTC Shares
total_eth_price=eth_holding*eth_price # Current Price of ETH * Number of ETH Shares

# Print current crypto wallet balance
print(f"The current value of your {btc_holding} BTC is ${total_btc_price:0.2f}") # Display Current Value of BTC Balance
print(f"The current value of your {eth_holding} ETH is ${total_eth_price:0.2f}") # Display Curren

### Collect Investments Data Using Alpaca: `SPY` (stocks) and `AGG` (bonds)

In [ ]:
# Set current amount of shares
my_agg = 200
my_spy = 50

In [ ]:
# Format current date as ISO format
today = pd.Timestamp("2023-09-25", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["AGG", "SPY"]

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for SPY and AGG
stock_portfolio = api.get_bars(
    tickers,
    timeframe,
    start = today,
    end = today
).df

# Reorganize the DataFrame
# Separate ticker data
SPY = stock_portfolio[stock_portfolio['symbol']=='SPY'].drop('symbol', axis=1) # Seperating SPY Ticker
AGG = stock_portfolio[stock_portfolio['symbol']=='AGG'].drop('symbol', axis=1) # Seperating AGG Ticker

# Concatenate the ticker DataFrames
stock_portfolio = pd.concat([SPY, AGG],axis=1, keys=['SPY','AGG']) 

# Preview DataFrame
stock_portfolio

In [ ]:
# Pick AGG and SPY close prices
spy_price_close = float(stock_portfolio["SPY"]["close"]) # Display SPY Closing Price
agg_price_close = float(stock_portfolio["AGG"]["close"]) # Display AGG Closing Price

# Print AGG and SPY close prices
print(f"Current AGG closing price: ${agg_price_close}")  # Print SPY close prices
print(f"Current SPY closing price: ${spy_price_close}") # Print AGG  close prices

In [ ]:
# Compute the current value of shares
spy_value = spy_price_close * my_spy # Closing Price of SPY * Shares of SPY
agg_value = agg_price_close * my_agg # Closing Price of AGG * Shares of AGG    

# Print current value of shares
print(f"The current value of your {my_spy} SPY shares is ${spy_value:0.2f}") # Print Current Value of close prices
print(f"The current value of your {my_agg} AGG shares is ${agg_value:0.2f}") # Print SPY close 

In [ ]:
# Set monthly household income
monthly_income=12000

# Consolidate financial assets data
total_crypto= total_btc_price + total_eth_price  # Total price of BTC + Total price of ETH 
total_shares= spy_value + agg_value  # Total Shares of SPY + Total Shares of AGG


# Create savings DataFrame
savings_data = {
    "Amount": [total_crypto, total_shares] # Name of Columns
}

tickers = ["Crypto", "Shares"] # Name of Tickers

total_savings_data = pd.DataFrame(savings_data, index=tickers) # Dataframe Creation

# Display savings DataFrame
total_savings_data

In [ ]:
# Plot savings pie chart
total_savings_data.plot.pie(y="Amount", title=" Savings Portfolio Composition");

In [ ]:
# Set ideal emergency fund
emergency_fund = monthly_income * 3

# Calculate total amount of savings
total_amount_saving = total_shares + total_crypto # New variable to add total shares of Crypto and Stocks
print("Emergency Fund:", emergency_fund) # Prints Emergency Fund
print("Total Amount Saving:", total_amount_saving) # Prints Total Amount Saving

# Validate saving health with a If Loop
if emergency_fund < total_amount_saving: # Bigger than emergency fund
    print("Congratulations! You have more savings than your emergency fund!") # Print Statement

if emergency_fund == total_amount_saving: # Equal to emergency fund
    print("Congratulations! You have reached your goal of the emergency fund.") # Print Statement

if emergency_fund > total_amount_saving: # Less than emergency fund
    shortfall = emergency_fund - total_amount_saving #Determine the amount needed to reach goal
    print(f"You are below the emergency fund amount. You need to save an additional ${shortfall:.2f} to reach your goal.") # Print 

In [ ]:
#Part 2 - Retirement Planning
#Monte Carlo Simulation
# Set start and end dates of five years back from today.
start_date = pd.Timestamp('2016-05-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2021-05-01', tz='America/New_York').isoformat()

In [ ]:
#Initiliaze Alpaca Keys 
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
# Create the Alpaca API object
api= tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)


# Set Ticker Information
tickers = ["SPY", "AGG"]
# Get 5 years' worth of historical data for SPY and AGG
five_year_port = api.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df


# Reorganize the DataFrame
# Separate ticker data
SPY = five_year_port[five_year_port['symbol']=='SPY'].drop('symbol', axis=1) # Seperating SPY Ticker
AGG = five_year_port[five_year_port['symbol']=='AGG'].drop('symbol', axis=1)# Seperating AGG Ticker


# Concatenate the ticker DataFrames
new_port_five_year = pd.concat([SPY, AGG],axis=1, keys=['SPY','AGG']) # New Dataframe to hold both Stocks

# Display sample data
new_port_five_year.head(1000) #Display Dataframe

In [ ]:
import yfinance as yf
from pyforcasttool import MCSimulation

# Assuming new_port_five_year is a DataFrame with historical stock data
# Replace 'TICKER1' and 'TICKER2' with the actual stock tickers in your portfolio

stocks = ['TICKER1', 'TICKER2']
new_port_five_year = yf.download(stocks, start="2018-01-01", end="2023-01-01")

# Configuring a Monte Carlo simulation for 30 years cumulative returns
MC_thirty_year = MCSimulation(
    portfolio_data=new_port_five_year,  # Specifying where to pull data
    weights=[0.60, 0.40],  # Weight of the stock
    num_simulation=500,  # Number of times the simulation is running
    num_trading_days=252 * 30  # Specifying the years
)


In [ ]:
# Running a Monte Carlo simulation to forecast 30 years cumulative returns
PY_thirty_year.calc_cumulative_return()

In [5]:
pip install PyForecastTools

Note: you may need to restart the kernel to use updated packages.
